# Armory-Library Tutorial

### In this tutorial, we will demonstrate how to run an example in armory-library with an example model and dataset. In this tutorial we will use the mnist dataset for simplicity and a model via HuggingFace that was trained on the mnist dataset.

### The necessary imports to run an armory library example
### We install datasets because we will use it later to retrieve a dataset from the HuggingFace hub. 

In [ ]:
!pip install armory-library
!pip install datasets

: 

### Imports for example
### These are all the necessary imports to run the example file and will all be used and explained later in the notebook

In [1]:
from pprint import pprint

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import datasets
import torch
import torch.nn
import torchmetrics.classification
from transformers import AutoImageProcessor, AutoModelForImageClassification

from armory.metrics.compute import BasicProfiler
from charmory.data import ArmoryDataLoader
from charmory.engine import EvaluationEngine
import charmory.evaluation as ev
from charmory.metrics.perturbation import PerturbationNormMetric
from charmory.model.image_classification import JaticImageClassificationModel
from charmory.perturbation import ArtEvasionAttack
from charmory.tasks.image_classification import ImageClassificationTask
from charmory.track import track_init_params, track_params
from charmory.utils import Unnormalize
import mlflow
from PIL import  Image
import os

2024-01-04 04:10:16.743910: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-04 04:10:16.782833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 04:10:16.782889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 04:10:16.783772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 04:10:16.789903: I tensorflow/core/platform/cpu_feature_guar

### We need to bring in the model and attach wrappers to run in this example

In [2]:
from art.defences.preprocessor import JpegCompression
import numpy as np


class JpegCompressionNormalized(JpegCompression):
    """
    Unnormalize inputs that were normalized during preprocessing,
    process use ART JpegCompression, and renormalize
    """

    def __init__(
        self,
        clip_values,
        quality=50,
        apply_fit=True,
        apply_predict=False,
        means=None,
        stds=None,
        dtype=np.float32,
    ):
        super().__init__(
            clip_values,
            quality=quality,
            apply_fit=apply_fit,
            apply_predict=apply_predict,
        )
        if means is None:
            means = (0.0, 0.0, 0.0)  # identity operation
        if len(means) != 3:
            raise ValueError("means must have 3 values, one per channel")
        self.means = np.array(means, dtype=dtype)

        if stds is None:
            stds = (1.0, 1.0, 1.0)  # identity operation
        if len(stds) != 3:
            raise ValueError("stds must have 3 values, one per channel")
        self.stds = np.array(stds, dtype=dtype)

    def __call__(self, x, y=None):
        x = x * self.stds + self.means
        x = torch.clamp(x, self.clip_values[0], self.clip_values[1], out=x)
        x, _ = super().__call__(x)
        x = (x - self.means) / self.stds
        return x, y





model = JaticImageClassificationModel(
    track_params(AutoModelForImageClassification.from_pretrained)(
        "farleyknight-org-username/vit-base-mnist"
    ),
    preadapter=JpegCompressionNormalized(
        clip_values=(0, 1),
        quality=50,
        means=(0.0, 0.0, 0.0),
        stds=(1.0, 1.0, 1.0),
    ),
)
classifier = track_init_params(PyTorchClassifier)(
    model,
    loss=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=0.003),
    input_shape=(3, 224, 224),
    channels_first=True,
    nb_classes=10,
    clip_values=(-1, 1),
)

For this section of code we create two variables `model` and `classifier`. We found a model on HuggingFace that is trained on the same mnist dataset with the model card `farleyknight-org-username/vit-base-mnist`. This can be replaced with another model off of Huggingface or you can use a custom local model.
- The model variable uses `AutoModelForImageClassification.from_pretrained` which takes in a HuggingFace model card name as a variable. This retrieves the model from
    HuggingFace that we will use for this example. `track_params` is a function wrapper that stores the argument values as parameters in MLflow. Lastly,
    the `JaticImageClassificationModel` is another wrapper to make the model compatible with Armory. This allows the model to have a standard output like other
    JATIC image classification models.
- The `PyTorchClassifier` class wraps the model to be usable by the ART library. It is specific to image classifier models written within the PyTorch framework. It takes in as arguments the model, loss function, and optimizer. The input image sizes are the shape of all the images inside the dataset. The `channels_first` variable is true because the images in the MNIST dataset are in a channels-first (C, H, W) multi-dimensional array. The `nb_classes` describe the number of classes model predicts on. Lastly the clip value  is the values that will be the min and max values of the input after scaling.We use `track_init_params` so that the constructor parameters for the ART wrapper are also tracked in MLflow.

### We need to get the dataset to use in this example.

In [3]:
    dataset = datasets.load_dataset("mnist", split="test")
    processor = AutoImageProcessor.from_pretrained(
        "farleyknight-org-username/vit-base-mnist"
    )

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.



For this section of code we create two variables `dataset` and `processor`.
- The `dataset` variable uses the datasets module directly from the HuggingFace API. The path location "mnist" is the location that of the same `dataset` that was used
  to train the model in this example. The parameter `split='test'` means that only the testing dataset is loading into the variable dataset. This is used because
  armory-library only using the testing dataset to test the adversarial robustness of the model.
- The `processor` variable obtains the model processor used by the model to use in our example. The is needed because the dataset needs to be in the correct form to be
  inputted into the model. The path passed into this function is a HuggingFace model card location that is the same location of the model in this example.  This function
  just obtains the `preprocessor`.



### The dataset needs to be preprocessed in the correct form for both armory-library and to be inputted into the model. A pytorch dataloader is needed to use the ART API correcly

In [4]:
batch_size = 16
def transform(sample):
        # Use the HF image processor and convert from BW To RGB
        sample["image"] = processor([img.convert("RGB") for img in sample["image"]])[
            "pixel_values"
        ]
        return sample

dataset.set_transform(transform)
dataloader = ArmoryDataLoader(dataset, batch_size=batch_size)


For this section of code we transform the `dataset` and create a `dataloader`.
- To transform the `dataset` we define a function `transform` to cycle through the `dataset` and convert each image into HuggingFace 'RGB' form. The transform function is
  using the HuggingFace API where you first create a function that cycles through a `dataset` and preforms the operation on the `dataset` that is needed to get it in correct
  form. Next the `set_transform` method is used by the HuggingFace dataset which applies the transform function to the entire HuggingFace dataset.
- The `dataloader` variable is created by using the `ArmoryDataLoader` function. The `ArmoryDataloader` is a custom PyTorch DataLoader that produces numpy arrays instead of
  Tensors which is required by ART. The only inputted used in this example is the `dataset` and batch size of images. All other variable can be passed in from the PyTorch
  DataLoader API.


## Next an attack is needed for armory-library that we will use to test the adversarial robustness of the machine learning model.

In [5]:
    attack = track_init_params(ProjectedGradientDescent)(
        classifier,
        batch_size=batch_size,
        eps=0.031,
        eps_step=0.007,
        max_iter=20,
        num_random_init=1,
        random_eps=False,
        targeted=False,
        verbose=False,
    )


For this section of code we create an `attack` variable.
- The `attack` variable is create with the `ProjectedGradientDescent` class which comes from the ART library. Here is a link to the paper it was modeled after
  https://arxiv.org/abs/1706.06083. The track_init_params is used to output the initial metrics to mlflow. It takes as an input the default specs for this `attack`, but
  these can be changed to be optimized to other models and datasets.


## A method is needed now to bring together the model, dataset, and attack that will execute all the code to test the adversarial robussness of the model.

In [6]:
evaluation = ev.Evaluation(
    name="mnist-vit-pgd",
    description="MNIST image classification using a ViT model and PGD attack",
    author="TwoSix",
    dataset=ev.Dataset(
        name="MNIST",
        x_key="image",
        y_key="label",
        test_dataloader=dataloader,
    ),
    model=ev.Model(
        name="ViT",
        model=classifier,
    ),
    perturbations={
        "benign": [],
        "attack": [
            ArtEvasionAttack(
                name="PGD",
                attack=attack,
                use_label_for_untargeted=False,
            ),
        ],
    },
    metric=ev.Metric(
        profiler=BasicProfiler(),
        perturbation={
            "linf_norm": PerturbationNormMetric(ord=torch.inf),
        },
        prediction={
            "accuracy": torchmetrics.classification.Accuracy(
                task="multiclass", num_classes=10
            ),
        },
    ),
)



For this section of code we create an `evalution` variable.
- The `evalution` variable is created by `ev.Evalution` class. This variable holds all the essential elements to run an adversarial robustness attack. The elements are:
```python
    name: 'user created name of the evalution (str)'
    description: 'description of the evalution (str)'
    model: ev.Model 
    dataset: ev.Dataset
    author: 'string of the author of the evalution (str)'
    perturbations: 'dictionary of names to lists of perturbations'
    metric:  ev.Metric
```

- The `model`, `dataset`, and `attack` were created in the earlier cells. The `metric` variable is using the basic standard metric function for image classification models in
  armory-library. In the `dataset` variable, the `x_key` and `y_key` need to be properly listed for the `dataset` being used. The `x_key` needs to be the column name of the image
  in the `dataset`. The `y_key` needs to be the column name of the label in the `dataset`.


### Some final detials are needed to output metrics to mlflow and the an engine class is needed to run the evalution class.

In [7]:
export_every_n_batches = 10
num_batches = 10
task = ImageClassificationTask(
        evaluation,
        export_adapter=Unnormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        export_every_n_batches=export_every_n_batches,
    )
engine = EvaluationEngine(task, limit_test_batches=num_batches)

INFO: Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs



For this section of code we create a `task` and `engine`:
- The task tells ART the type of model the attack is running on. Armory-library currently has two tasks image classification and object detection. It takes the `evalution`
  variable, number of classes in the model, export adapter, and `export_every_n_batches`. The `export_every_n_batches` variable is the amount of batches that get exported to mlFlow, so if
  10 is selected then every 10 batches are outputted on mlFlow. The adapter takes in the Unnormalize variable which preforms the inverse of the PyTorch Normalize function. This
  operation allows the normalize dataset to be reversed to allow the unnormalized data to be displayed in mlflow.
- The `engine` variable is created from the `Evaluation` Engine which takes the task and `limit_test_batches` as variables. The `limit_test_batches` is the amount of batches of testing
  data that the engine will create metrics for. The `Evaluation` Engine only has one method `.run()` which runs the experiment.


#### Lastly, the engine variable executes .run() which runs the experiment. Function pprint is used to print the output of the run in a better format than regular python print.

In [8]:
pprint(engine.run())

2024/01/04 04:10:22 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/01/04 04:10:22 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/jonathan.prokos/.venv/armory-library/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      attack/accuracy      │    0.02500000037252903    │
│     attack/linf_norm      │    0.03100001811981201    │
│      benign/accuracy      │            1.0            │
│     benign/linf_norm      │            0.0            │
└───────────────────────────┴───────────────────────────┘

2024/01/04 04:11:03 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/01/04 04:11:03 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


{'compute': {'Avg. CPU time (s) for 10 executions of attack/perturbation': 3.7299391730921343,
             'Avg. CPU time (s) for 10 executions of attack/perturbation/PGD': 3.729918852308765,
             'Avg. CPU time (s) for 10 executions of attack/predict': 0.10566661852644757,
             'Avg. CPU time (s) for 10 executions of benign/perturbation': 1.118588261306286e-06,
             'Avg. CPU time (s) for 10 executions of benign/predict': 0.1220854700775817},
 'metrics': {'attack/accuracy': tensor(0.0250),
             'attack/linf_norm': tensor(0.0310),
             'benign/accuracy': tensor(1.),
             'benign/linf_norm': tensor(0.)}}


In [9]:
#Simple script to retrieve 1 example of a benign and attacked image from the mlflow artifacts
mlflow_var = mlflow.search_experiments(filter_string="name = 'mnist-vit-pgd'")[0]
path = mlflow_var.artifact_location[7:] + '/' + engine.run_id + '/artifacts'
image_bin, image_adv = None, None
for f in os.listdir(path):
    if f.endswith("ex_0_x_adv.png"):
        image_adv = Image.open(os.path.join(path,f))
    elif f.endswith("ex_0_x.png"):
        image_bin = Image.open(os.path.join(path,f))


## Benign Image

In [10]:
image_bin

## Attacked Image

In [11]:
image_adv